# Joining the Data

It is now time for the scary part. But we never back away from a challenge! Let's say a prayer and get ready to join the data.

In [1]:
import pandas as pd
import numpy as np

# Merge 1

### Loading in Utility Scale Net Generation Data and Reliability of Distribution Systems Date 

In [2]:
net_gen = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Utility_Scale_Net_Generation.csv', index_col=0)

net_gen.head()



,State,Total_Generation_Rank,Electric_Generation_Rank,Commercial_Generation_Rank,Capacity_Growth_Status
0,CT,Moderate,Low,Moderate,Growing
2,MA,Low,Low,High,Growing
4,RI,Low,Low,Low,Shrinking
5,VT,Low,Low,Low,Shrinking
6,NJ,Moderate,Low,High,Shrinking


In [3]:
net_gen['State'].value_counts().sum()

39

In [4]:
reliability = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Reliability_Distribution.csv', index_col=0)

reliability.head()

,Census Division,State,Percent of Customers Reported w/ Major Events,SAIDI w/ Major Events,SAIFI w/ Major Events,CAIDI w/ Major Events,SAIDI w/o Major Events,SAIFI w/o Major Events,CAIDI w/o Major Events
0,New England,CT,100.6,164.6,0.9,188.8,70.3,0.7,107.3
1,New England,ME,100.8,1863.0,3.3,562.2,247.4,2.0,125.0
2,New England,MA,88.2,259.4,1.1,245.4,82.2,0.8,105.4
3,New England,NH,99.3,645.8,1.8,368.5,124.6,1.0,126.1
4,New England,RI,98.3,104.6,0.8,133.7,52.2,0.7,78.3


In [5]:
reliability.corr(numeric_only=True)

,Percent of Customers Reported w/ Major Events,SAIDI w/ Major Events,SAIFI w/ Major Events,CAIDI w/ Major Events,SAIDI w/o Major Events,SAIFI w/o Major Events,CAIDI w/o Major Events
Percent of Customers Reported w/ Major Events,1.000000,0.155055,-0.011626,0.245667,0.005549,-0.077099,0.106543
SAIDI w/ Major Events,0.155055,1.000000,0.832496,0.887799,0.652470,0.644317,0.427449
SAIFI w/ Major Events,-0.011626,0.832496,1.000000,0.573910,0.859044,0.939123,0.365608
CAIDI w/ Major Events,0.245667,0.887799,0.573910,1.000000,0.438757,0.340570,0.490745
SAIDI w/o Major Events,0.005549,0.652470,0.859044,0.438757,1.000000,0.911040,0.678920
SAIFI w/o Major Events,-0.077099,0.644317,0.939123,0.340570,0.911040,1.000000,0.357660
CAIDI w/o Major Events,0.106543,0.427449,0.365608,0.490745,0.678920,0.357660,1.000000


In [6]:
# Drop highly correlated metrics
reliability = reliability.drop(columns=['SAIDI w/ Major Events', 'SAIDI w/o Major Events', 'SAIFI w/o Major Events', 'Percent of Customers Reported w/ Major Events', 'Census Division'])
reliability.head()

,State,SAIFI w/ Major Events,CAIDI w/ Major Events,CAIDI w/o Major Events
0,CT,0.9,188.8,107.3
1,ME,3.3,562.2,125.0
2,MA,1.1,245.4,105.4
3,NH,1.8,368.5,126.1
4,RI,0.8,133.7,78.3


In [7]:
reliability.describe()

,SAIFI w/ Major Events,CAIDI w/ Major Events,CAIDI w/o Major Events
count,51.000000,51.000000,51.000000
mean,1.386275,226.466667,116.519608
std,0.636873,127.799489,23.595296
min,0.400000,76.200000,70.800000
25%,0.900000,134.850000,101.350000
50%,1.100000,186.900000,118.000000
75%,1.800000,282.200000,124.950000
max,3.300000,723.500000,206.200000


In [8]:
# Bin SAIFI (Average Interruption Frequency Index) w/ Major Events based on industry standards
conditions_saifi = [
    reliability['SAIFI w/ Major Events'] > 2.0,  # Poor reliability: More than 2 interruptions per year
    (reliability['SAIFI w/ Major Events'] >= 1.0) & (reliability['SAIFI w/ Major Events'] <= 2.0),  # Average reliability: 1-2 interruptions
    reliability['SAIFI w/ Major Events'] < 1.0  # Good reliability: Less than 1 interruption per year
]
choices_saifi = ['High', 'Moderate', 'Low']
reliability['SAIFI Major Events'] = np.select(conditions_saifi, choices_saifi, default='Unknown')

In [9]:
reliability.head()

,State,SAIFI w/ Major Events,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events
0,CT,0.9,188.8,107.3,Low
1,ME,3.3,562.2,125.0,High
2,MA,1.1,245.4,105.4,Moderate
3,NH,1.8,368.5,126.1,Moderate
4,RI,0.8,133.7,78.3,Low


In [10]:
reliability = reliability.drop(columns=['SAIFI w/ Major Events'])
reliability.head()

,State,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events
0,CT,188.8,107.3,Low
1,ME,562.2,125.0,High
2,MA,245.4,105.4,Moderate
3,NH,368.5,126.1,Moderate
4,RI,133.7,78.3,Low


In [11]:
# Count duplicates of State
reliability['State'].value_counts().sum()

51

In [12]:
# Conduct an outer join on the State column
net_gen_reliability = pd.merge(net_gen, reliability, on='State', how='outer')

net_gen_reliability.head()

,State,Total_Generation_Rank,Electric_Generation_Rank,Commercial_Generation_Rank,Capacity_Growth_Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events
0,AK,Low,Low,High,Growing,138.8,137.6,High
1,AL,High,High,Low,Growing,186.9,122.8,Moderate
2,AR,NaN,NaN,NaN,NaN,433.4,144.0,High
3,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate
4,CA,High,High,High,Shrinking,248.1,140.0,Moderate


In [13]:
net_gen_reliability.isna().value_counts()

State  Total_Generation_Rank  Electric_Generation_Rank  Commercial_Generation_Rank  Capacity_Growth_Status  CAIDI w/ Major Events  CAIDI w/o Major Events  SAIFI Major Events
False  False                  False                     False                       False                   False                  False                   False                 39
       True                   True                      True                        True                    False                  False                   False                 12
Name: count, dtype: int64

In [14]:
net_gen_reliability.fillna('Unknown', inplace=True)
net_gen_reliability.head()

,State,Total_Generation_Rank,Electric_Generation_Rank,Commercial_Generation_Rank,Capacity_Growth_Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events
0,AK,Low,Low,High,Growing,138.8,137.6,High
1,AL,High,High,Low,Growing,186.9,122.8,Moderate
2,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High
3,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate
4,CA,High,High,High,Shrinking,248.1,140.0,Moderate


## Load in the Energy Efficiency data

In [15]:
energy_efficiency = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Efficiency.csv', index_col=0)

energy_efficiency.head()

,Utility Number,Utility Name,State,BA Code,Total Annual Savings (MWh),Total Peak Demand Annual Savings (MW),Total Annual Incent Cost (Thousand $),Total Non-Incentive Annual Costs (Thousand $),Commercial Weighted Avg Life (Yrs)
2,195,Alabama Power Co,AL,SOCO,5767.0,11.9,1225.0,1095.0,5.000
3,207,Alameda Municipal Power,CA,CISO,146.0,0.1,70.0,321.0,13.269
4,295,City of Alexandria - (MN),MN,MISO,3306.0,0.7,187.0,194.0,12.727
5,554,City of Ames - (IA),IA,MISO,189.0,0.1,197.0,54.0,10.000
6,590,City of Anaheim - (CA),CA,CISO,7242.0,1.5,3338.0,804.0,11.437


In [16]:
energy_efficiency.columns

Index(['Utility Number', 'Utility Name', 'State', 'BA Code',
       'Total Annual Savings (MWh)', 'Total Peak Demand Annual Savings (MW)',
       'Total Annual Incent Cost (Thousand $)',
       'Total Non-Incentive Annual Costs (Thousand $)',
       'Commercial Weighted Avg Life (Yrs)'],
      dtype='object')

In [17]:
energy_efficiency.corr(numeric_only=True)

,Utility Number,Total Annual Savings (MWh),Total Peak Demand Annual Savings (MW),Total Annual Incent Cost (Thousand $),Total Non-Incentive Annual Costs (Thousand $),Commercial Weighted Avg Life (Yrs)
Utility Number,1.000000,0.021878,0.025283,0.148453,0.083285,0.001019
Total Annual Savings (MWh),0.021878,1.000000,0.473292,0.639695,0.812012,0.112121
Total Peak Demand Annual Savings (MW),0.025283,0.473292,1.000000,0.278101,0.402326,0.037236
Total Annual Incent Cost (Thousand $),0.148453,0.639695,0.278101,1.000000,0.600819,0.024366
Total Non-Incentive Annual Costs (Thousand $),0.083285,0.812012,0.402326,0.600819,1.000000,0.018572
Commercial Weighted Avg Life (Yrs),0.001019,0.112121,0.037236,0.024366,0.018572,1.000000


In [18]:
# Drop the year and Utility number columns
energy_efficiency = energy_efficiency.drop(columns=['Utility Number', 'Utility Name','Total Non-Incentive Annual Costs (Thousand $)', 'BA Code'])

energy_efficiency.head()

,State,Total Annual Savings (MWh),Total Peak Demand Annual Savings (MW),Total Annual Incent Cost (Thousand $),Commercial Weighted Avg Life (Yrs)
2,AL,5767.0,11.9,1225.0,5.000
3,CA,146.0,0.1,70.0,13.269
4,MN,3306.0,0.7,187.0,12.727
5,IA,189.0,0.1,197.0,10.000
6,CA,7242.0,1.5,3338.0,11.437


In [19]:
energy_efficiency.describe()

,Total Annual Savings (MWh),Total Peak Demand Annual Savings (MW),Total Annual Incent Cost (Thousand $),Commercial Weighted Avg Life (Yrs)
count,3.280000e+02,328.000000,328.000000,328.000000
mean,7.298820e+04,17.305183,11393.076220,12.505122
std,1.879049e+05,62.953090,32636.461121,4.285681
min,1.200000e+01,0.000000,0.000000,0.000000
25%,1.214500e+03,0.200000,190.250000,11.000000
50%,5.769500e+03,1.500000,1109.000000,12.700000
75%,5.768900e+04,11.450000,7041.750000,14.312500
max,1.718664e+06,945.000000,372401.000000,27.525000


** Note that MW measures power capacity while MWh measures the amount of electricity delivered overtime

In [20]:
# Bin Total Annual Savings (MWh) based on DOE program size classifications
conditions_savings = [
    energy_efficiency['Total Annual Savings (MWh)'] >= 100000,  # Large utility programs (>100 GWh)
    (energy_efficiency['Total Annual Savings (MWh)'] >= 25000) & 
    (energy_efficiency['Total Annual Savings (MWh)'] < 100000),  # Medium programs (25-100 GWh)
    energy_efficiency['Total Annual Savings (MWh)'] < 25000  # Small programs (<25 GWh)
]
choices_savings = ['High', 'Moderate', 'Low']
energy_efficiency['Overall Annual Energy Savings (MWh)'] = np.select(conditions_savings, choices_savings, default='Unknown')

# Bin Peak Demand Savings based on utility demand response program standards
conditions_peak = [
    energy_efficiency['Total Peak Demand Annual Savings (MW)'] >= 50,  # Large DR programs (>50 MW)
    (energy_efficiency['Total Peak Demand Annual Savings (MW)'] >= 10) & 
    (energy_efficiency['Total Peak Demand Annual Savings (MW)'] < 50),  # Medium DR programs (10-50 MW)
    energy_efficiency['Total Peak Demand Annual Savings (MW)'] < 10  # Small DR programs (<10 MW)
]
choices_peak = ['High', 'Moderate', 'Low']
energy_efficiency['Overall Peak Demand Savings (MW)'] = np.select(conditions_peak, choices_peak, default='Unknown')

In [21]:
energy_efficiency.describe()

,Total Annual Savings (MWh),Total Peak Demand Annual Savings (MW),Total Annual Incent Cost (Thousand $),Commercial Weighted Avg Life (Yrs)
count,3.280000e+02,328.000000,328.000000,328.000000
mean,7.298820e+04,17.305183,11393.076220,12.505122
std,1.879049e+05,62.953090,32636.461121,4.285681
min,1.200000e+01,0.000000,0.000000,0.000000
25%,1.214500e+03,0.200000,190.250000,11.000000
50%,5.769500e+03,1.500000,1109.000000,12.700000
75%,5.768900e+04,11.450000,7041.750000,14.312500
max,1.718664e+06,945.000000,372401.000000,27.525000


In [22]:
# Bin Total Annual Incent Cost using fixed thresholds (in thousands of dollars)
conditions_incent = [
    energy_efficiency['Total Annual Incent Cost (Thousand $)'] >= 10000,  # Very large programs (>$10M)
    (energy_efficiency['Total Annual Incent Cost (Thousand $)'] >= 2000) & 
    (energy_efficiency['Total Annual Incent Cost (Thousand $)'] < 10000),  # Mid-size programs ($2-10M) 
    energy_efficiency['Total Annual Incent Cost (Thousand $)'] < 2000  # Smaller programs (<$2M)
]
choices_incent = ['High', 'Moderate', 'Low']
energy_efficiency['Overall Annual Incentive Cost'] = np.select(conditions_incent, choices_incent, default='Unknown')

# Bin Commercial Weighted Avg Life using fixed thresholds (in years)
conditions_life = [
    energy_efficiency['Commercial Weighted Avg Life (Yrs)'] >= 20,  # Long-life equipment (HVAC, building shell)
    (energy_efficiency['Commercial Weighted Avg Life (Yrs)'] >= 12) & 
    (energy_efficiency['Commercial Weighted Avg Life (Yrs)'] < 20),  # Medium-life equipment (lighting, motors)
    energy_efficiency['Commercial Weighted Avg Life (Yrs)'] < 12  # Short-life equipment (controls, maintenance)
]
choices_life = ['High', 'Moderate', 'Low']
energy_efficiency['Overall Commercial Avg Life (Yrs)'] = np.select(conditions_life, choices_life, default='Unknown')


In [23]:
energy_efficiency.head()

,State,Total Annual Savings (MWh),Total Peak Demand Annual Savings (MW),Total Annual Incent Cost (Thousand $),Commercial Weighted Avg Life (Yrs),Overall Annual Energy Savings (MWh),Overall Peak Demand Savings (MW),Overall Annual Incentive Cost,Overall Commercial Avg Life (Yrs)
2,AL,5767.0,11.9,1225.0,5.000,Low,Moderate,Low,Low
3,CA,146.0,0.1,70.0,13.269,Low,Low,Low,Moderate
4,MN,3306.0,0.7,187.0,12.727,Low,Low,Low,Moderate
5,IA,189.0,0.1,197.0,10.000,Low,Low,Low,Low
6,CA,7242.0,1.5,3338.0,11.437,Low,Low,Moderate,Low


In [24]:
# Keep only relevant columns
energy_efficiency_clean = energy_efficiency[['State', 'Overall Annual Energy Savings (MWh)',
                                            'Overall Peak Demand Savings (MW)', 'Overall Annual Incentive Cost', 'Overall Commercial Avg Life (Yrs)']]

energy_efficiency_clean.head()

,State,Overall Annual Energy Savings (MWh),Overall Peak Demand Savings (MW),Overall Annual Incentive Cost,Overall Commercial Avg Life (Yrs)
2,AL,Low,Moderate,Low,Low
3,CA,Low,Low,Low,Moderate
4,MN,Low,Low,Low,Moderate
5,IA,Low,Low,Low,Low
6,CA,Low,Low,Moderate,Low


In [25]:
# Rename columns for more clarity
energy_efficiency_clean = energy_efficiency_clean.rename(columns={'Overall Annual Energy Savings (MWh)': 'Utility Annual Energy Savings (MWh)',
                                                                  'Overall Peak Demand Savings (MW)':'Utility Peak Demand Savings (MW)',
                                                                  'Overall Annual Incentive Cost': 'Utility Annual Incentive Cost ($)',
                                                                  'Overall Commercial Avg Life (Yrs)': 'Utility Commercial Avg Life (Yrs)'})
energy_efficiency_clean.head()

,State,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs)
2,AL,Low,Moderate,Low,Low
3,CA,Low,Low,Low,Moderate
4,MN,Low,Low,Low,Moderate
5,IA,Low,Low,Low,Low
6,CA,Low,Low,Moderate,Low


In [26]:
energy_efficiency_clean.isna().value_counts()

State  Utility Annual Energy Savings (MWh)  Utility Peak Demand Savings (MW)  Utility Annual Incentive Cost ($)  Utility Commercial Avg Life (Yrs)
False  False                                False                             False                              False                                328
Name: count, dtype: int64

In [27]:
len(energy_efficiency_clean['State'].value_counts())

48

In [28]:
# Merge net_gen_reliability and energy_efficiency on the State column
merged_2 = pd.merge(net_gen_reliability, energy_efficiency_clean, on='State', how='outer')

merged_2.head()

,State,Total_Generation_Rank,Electric_Generation_Rank,Commercial_Generation_Rank,Capacity_Growth_Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs)
0,AK,Low,Low,High,Growing,138.8,137.6,High,NaN,NaN,NaN,NaN
1,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low
2,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate
3,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,High,Moderate,High,Moderate
4,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,Moderate,Low,Moderate,Moderate


In [29]:
merged_2.rename(columns={'Total_Generation_Rank': 'Total Generation Rank', 'Electric_Generation_Rank': 'Electric Generation Rank', 
                         'Commercial_Generation_Rank': 'Commercial Generation Rank','Capacity_Growth_Status': 'Capacity Growth Status'}, inplace=True)

In [30]:
merged_2.isna().value_counts()

State  Total Generation Rank  Electric Generation Rank  Commercial Generation Rank  Capacity Growth Status  CAIDI w/ Major Events  CAIDI w/o Major Events  SAIFI Major Events  Utility Annual Energy Savings (MWh)  Utility Peak Demand Savings (MW)  Utility Annual Incentive Cost ($)  Utility Commercial Avg Life (Yrs)
False  False                  False                     False                       False                   False                  False                   False               False                                False                             False                              False                                328
                                                                                                                                                                               True                                 True                              True                               True                                   3
Name: count, dtype: int64

In [31]:
merged_2.isin(['Unknown']).sum()

State                                   0
Total Generation Rank                  52
Electric Generation Rank               52
Commercial Generation Rank             52
Capacity Growth Status                 52
CAIDI w/ Major Events                   0
CAIDI w/o Major Events                  0
SAIFI Major Events                      0
Utility Annual Energy Savings (MWh)     0
Utility Peak Demand Savings (MW)        0
Utility Annual Incentive Cost ($)       0
Utility Commercial Avg Life (Yrs)       0
dtype: int64

### Loading in the regional Demand data for joining

In [32]:
demand = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Regional_Demand_2023.csv', index_col=0)
demand.head()

,Region,State,Annual_Region_Demand_MWh
0,CAL,CA,726482.230137
1,CAL,CA,726482.230137
2,CAL,CA,726482.230137
3,CAL,CA,726482.230137
4,CAL,CA,726482.230137


In [33]:
demand.describe()

,Annual_Region_Demand_MWh
count,1.679000e+04
mean,1.029680e+06
std,6.637723e+05
min,3.068565e+05
25%,5.984282e+05
50%,7.698936e+05
75%,1.920122e+06
max,2.150311e+06


In [34]:
# Bin Annual Region Demand using fixed thresholds (in MWh)
conditions_demand = [
    demand['Annual_Region_Demand_MWh'] >= 1500000,  # Very large regions (>1.5M MWh) like PJM, ERCOT
    (demand['Annual_Region_Demand_MWh'] >= 750000) & 
    (demand['Annual_Region_Demand_MWh'] < 1500000),  # Mid-size regions (750k-1.5M MWh) like NYISO, CAISO
    demand['Annual_Region_Demand_MWh'] < 750000  # Smaller regions (<750k MWh) like ISO-NE, SPP
]
choices_demand = ['High', 'Moderate', 'Low']
demand['Regional Electricity Demand'] = np.select(conditions_demand, choices_demand, default='Unknown')

In [35]:
demand = demand.drop(columns=['Annual_Region_Demand_MWh'])

In [36]:
demand.shape

(16790, 3)

In [37]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16790 entries, 0 to 16789
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Region                       16790 non-null  object
 1   State                        16790 non-null  object
 2   Regional Electricity Demand  16790 non-null  object
dtypes: object(3)
memory usage: 524.7+ KB


In [38]:
demand[demand.duplicated(subset=['Region', 'State'])]

,Region,State,Regional Electricity Demand
1,CAL,CA,Low
2,CAL,CA,Low
3,CAL,CA,Low
4,CAL,CA,Low
5,CAL,CA,Low
...,...,...,...
16785,TEX,TX,Moderate
16786,TEX,TX,Moderate
16787,TEX,TX,Moderate
16788,TEX,TX,Moderate


In [39]:
# Drop duplicates of region and state
demand = demand.drop_duplicates(subset=['Region', 'State'])

# Verify the result
print(demand.shape)
demand.info()

(46, 3)
<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 0 to 16425
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Region                       46 non-null     object
 1   State                        46 non-null     object
 2   Regional Electricity Demand  46 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


In [40]:
demand.drop(columns=['Region'], inplace=True)

In [41]:
# Merge demand with merged 2

merged_3 = pd.merge(merged_2, demand, on='State', how='outer')
merged_3.head()

,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand
0,AK,Low,Low,High,Growing,138.8,137.6,High,NaN,NaN,NaN,NaN,Moderate
1,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low
2,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low
3,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,High,Moderate,High,Moderate,Low
4,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,Moderate,Low,Moderate,Moderate,Low


In [42]:
merged_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                339 non-null    object 
 1   Total Generation Rank                339 non-null    object 
 2   Electric Generation Rank             339 non-null    object 
 3   Commercial Generation Rank           339 non-null    object 
 4   Capacity Growth Status               339 non-null    object 
 5   CAIDI w/ Major Events                339 non-null    float64
 6   CAIDI w/o Major Events               339 non-null    float64
 7   SAIFI Major Events                   339 non-null    object 
 8   Utility Annual Energy Savings (MWh)  336 non-null    object 
 9   Utility Peak Demand Savings (MW)     336 non-null    object 
 10  Utility Annual Incentive Cost ($)    336 non-null    object 
 11  Utility Commercial Avg Life (Yrs

In [43]:
merged_3[merged_3['Regional Electricity Demand'].isna()]

,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand
37,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
38,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
39,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Moderate,Low,Moderate,Low,NaN
40,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
41,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
42,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
43,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
44,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
45,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN
46,CO,Moderate,Moderate,Low,Shrinking,108.3,102.2,Low,Low,Low,Low,Moderate,NaN


In [44]:
merged_3['Regional Electricity Demand'].fillna('Unknown', inplace=True)
merged_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                339 non-null    object 
 1   Total Generation Rank                339 non-null    object 
 2   Electric Generation Rank             339 non-null    object 
 3   Commercial Generation Rank           339 non-null    object 
 4   Capacity Growth Status               339 non-null    object 
 5   CAIDI w/ Major Events                339 non-null    float64
 6   CAIDI w/o Major Events               339 non-null    float64
 7   SAIFI Major Events                   339 non-null    object 
 8   Utility Annual Energy Savings (MWh)  336 non-null    object 
 9   Utility Peak Demand Savings (MW)     336 non-null    object 
 10  Utility Annual Incentive Cost ($)    336 non-null    object 
 11  Utility Commercial Avg Life (Yrs

/var/folders/rj/3yvmqxnj3d17zpgh8wpywxvr0000gn/T/ipykernel_11476/1737813523.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_3['Regional Electricity Demand'].fillna('Unknown', inplace=True)


In [45]:
merged_3.fillna('Unknown', inplace=True)
merged_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                339 non-null    object 
 1   Total Generation Rank                339 non-null    object 
 2   Electric Generation Rank             339 non-null    object 
 3   Commercial Generation Rank           339 non-null    object 
 4   Capacity Growth Status               339 non-null    object 
 5   CAIDI w/ Major Events                339 non-null    float64
 6   CAIDI w/o Major Events               339 non-null    float64
 7   SAIFI Major Events                   339 non-null    object 
 8   Utility Annual Energy Savings (MWh)  339 non-null    object 
 9   Utility Peak Demand Savings (MW)     339 non-null    object 
 10  Utility Annual Incentive Cost ($)    339 non-null    object 
 11  Utility Commercial Avg Life (Yrs

In [46]:
ixp = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_IXP.csv', index_col=0)
ixp.head()

,IXP Name,City,State
0,48 IX,Phoenix,AZ
1,ABQIX,Albuquerque,NM
2,AlaskaIX,Anchorage,AK
3,Amateur Radio Internet Exchange - ARIX,Fremont,CA
4,Amateur Radio Internet Exchange - ARIX,Portland,OR


In [47]:
ixp.isna().value_counts()

IXP Name  City   State
False     False  False    196
                 True       3
Name: count, dtype: int64

In [48]:
ixp[ixp['State'].isna()]

,IXP Name,City,State
21,BroadBand Internet Exchange US-West - BBIX US-...,West Coast,NaN
32,CoreSite - Any2East,Washington DC,NaN
74,Equinix Internet Exchange Seattle - Equinix Se...,Seattle,NaN


In [49]:
ixp.loc[(ixp['City'] == 'Seattle') & (ixp['State'] == 'Washington'), 'City'] = 'Washington'

In [50]:
# Fix Seattle, Washington data
ixp.loc[ixp['City'] == 'Seattle', 'State'] = 'WA'

# Fix Washington DC data
ixp.loc[ixp['City'] == 'Washington DC', ['City', 'State']] = ['Washington', 'DC']

#Drop rows where City is West Coast
ixp = ixp[ixp['City'] != 'West Coast']

In [51]:
ixp[ixp['City'] == 'Seattle']

,IXP Name,City,State
74,Equinix Internet Exchange Seattle - Equinix Se...,Seattle,WA
123,Megaport MegaIX Seattle,Seattle,WA
132,Moe Internet Exchange - Seattle - MoeIX SEA,Seattle,WA
160,Pacific Wave Exchange in LA,Seattle,WA
171,Seattle Internet Exchange (MTU 1500) - SIX Sea...,Seattle,WA
172,Seattle Internet Exchange (MTU 9000) - SIX Sea...,Seattle,WA


In [52]:
ixp[ixp['City'] == 'Washington']

,IXP Name,City,State
32,CoreSite - Any2East,Washington,DC
85,Global Peer Exchange,Washington,DC


In [53]:
len(ixp['State'].value_counts())

42

In [54]:
len(ixp['City'].value_counts())

92

In [55]:
ixp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 198 entries, 0 to 198
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IXP Name  198 non-null    object
 1   City      198 non-null    object
 2   State     198 non-null    object
dtypes: object(3)
memory usage: 6.2+ KB


In [56]:
# Merge ixp data
merged_4 = pd.merge(merged_3, ixp, on='State', how='outer')

merged_4.isna().value_counts()

State  Total Generation Rank  Electric Generation Rank  Commercial Generation Rank  Capacity Growth Status  CAIDI w/ Major Events  CAIDI w/o Major Events  SAIFI Major Events  Utility Annual Energy Savings (MWh)  Utility Peak Demand Savings (MW)  Utility Annual Incentive Cost ($)  Utility Commercial Avg Life (Yrs)  Regional Electricity Demand  IXP Name  City 
False  False                  False                     False                       False                   False                  False                   False               False                                False                             False                              False                              False                        False     False    2251
                                                                                                                                                                                                                                                                              

In [57]:
merged_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                2291 non-null   object 
 1   Total Generation Rank                2290 non-null   object 
 2   Electric Generation Rank             2290 non-null   object 
 3   Commercial Generation Rank           2290 non-null   object 
 4   Capacity Growth Status               2290 non-null   object 
 5   CAIDI w/ Major Events                2290 non-null   float64
 6   CAIDI w/o Major Events               2290 non-null   float64
 7   SAIFI Major Events                   2290 non-null   object 
 8   Utility Annual Energy Savings (MWh)  2290 non-null   object 
 9   Utility Peak Demand Savings (MW)     2290 non-null   object 
 10  Utility Annual Incentive Cost ($)    2290 non-null   object 
 11  Utility Commercial Avg Life (Y

In [58]:
merged_4[merged_4['City'].isna()]

,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP Name,City
5,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,High,Moderate,High,Moderate,Low,NaN,NaN
6,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,Moderate,Low,Moderate,Moderate,Low,NaN,NaN
7,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,Moderate,Moderate,Moderate,Moderate,Low,NaN,NaN
903,DE,Unknown,Unknown,Unknown,Unknown,130.3,87.3,Low,Low,Moderate,Low,Low,High,NaN,NaN
904,DE,Unknown,Unknown,Unknown,Unknown,130.3,87.3,Low,Low,Low,Low,Moderate,High,NaN,NaN
1129,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Low,Moderate,NaN,NaN
1130,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Low,Moderate,NaN,NaN
1131,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,High,Moderate,High,Moderate,Moderate,NaN,NaN
1132,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Moderate,Moderate,NaN,NaN
1133,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Moderate,Moderate,NaN,NaN


In [59]:
merged_4['IXP Name'] = merged_4['IXP Name'].fillna('No IXP')

In [60]:
merged_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                2291 non-null   object 
 1   Total Generation Rank                2290 non-null   object 
 2   Electric Generation Rank             2290 non-null   object 
 3   Commercial Generation Rank           2290 non-null   object 
 4   Capacity Growth Status               2290 non-null   object 
 5   CAIDI w/ Major Events                2290 non-null   float64
 6   CAIDI w/o Major Events               2290 non-null   float64
 7   SAIFI Major Events                   2290 non-null   object 
 8   Utility Annual Energy Savings (MWh)  2290 non-null   object 
 9   Utility Peak Demand Savings (MW)     2290 non-null   object 
 10  Utility Annual Incentive Cost ($)    2290 non-null   object 
 11  Utility Commercial Avg Life (Y

In [61]:
merged_4[merged_4['Total Generation Rank'].isna()]

,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP Name,City
2099,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DartNode IXP - DartNode IXP - Houston,Houston


In [62]:
# Drop rows where Total Generation Rank is NaN
merged_4 = merged_4.dropna(subset=['Total Generation Rank'])
merged_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2290 entries, 0 to 2290
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                2290 non-null   object 
 1   Total Generation Rank                2290 non-null   object 
 2   Electric Generation Rank             2290 non-null   object 
 3   Commercial Generation Rank           2290 non-null   object 
 4   Capacity Growth Status               2290 non-null   object 
 5   CAIDI w/ Major Events                2290 non-null   float64
 6   CAIDI w/o Major Events               2290 non-null   float64
 7   SAIFI Major Events                   2290 non-null   object 
 8   Utility Annual Energy Savings (MWh)  2290 non-null   object 
 9   Utility Peak Demand Savings (MW)     2290 non-null   object 
 10  Utility Annual Incentive Cost ($)    2290 non-null   object 
 11  Utility Commercial Avg Life (Yrs)  

In [63]:
merged_4.loc[merged_4['State'] == 'Texas', 'State'] = 'TX'

merged_4['State'].value_counts()

State
CA    780
TX    294
FL    140
WA    119
NY    110
OR    105
CO     65
MO     64
MA     64
MI     57
IL     55
IA     48
MN     42
AZ     40
VA     39
NC     32
OH     30
GA     30
PA     24
WI     14
NE     14
MD     10
UT      9
IN      8
NV      8
CT      8
SD      6
VT      6
ID      6
HI      6
NJ      5
KY      5
SC      5
AL      4
LA      4
NH      4
NM      4
MS      4
WY      3
AR      3
RI      2
OK      2
MT      2
DE      2
DC      2
ND      1
TN      1
ME      1
KS      1
WV      1
AK      1
Name: count, dtype: int64

In [64]:
merged_4[merged_4['City'].isna()]

,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP Name,City
5,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,High,Moderate,High,Moderate,Low,No IXP,NaN
6,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,Moderate,Low,Moderate,Moderate,Low,No IXP,NaN
7,AR,Unknown,Unknown,Unknown,Unknown,433.4,144.0,High,Moderate,Moderate,Moderate,Moderate,Low,No IXP,NaN
903,DE,Unknown,Unknown,Unknown,Unknown,130.3,87.3,Low,Low,Moderate,Low,Low,High,No IXP,NaN
904,DE,Unknown,Unknown,Unknown,Unknown,130.3,87.3,Low,Low,Low,Low,Moderate,High,No IXP,NaN
1129,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Low,Moderate,No IXP,NaN
1130,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Low,Moderate,No IXP,NaN
1131,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,High,Moderate,High,Moderate,Moderate,No IXP,NaN
1132,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Moderate,Moderate,No IXP,NaN
1133,ID,Low,Moderate,Low,Shrinking,128.4,121.2,Moderate,Low,Low,Low,Moderate,Moderate,No IXP,NaN


In [65]:
merged_4 = merged_4.dropna(subset=['City'])
merged_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2251 entries, 0 to 2287
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                2251 non-null   object 
 1   Total Generation Rank                2251 non-null   object 
 2   Electric Generation Rank             2251 non-null   object 
 3   Commercial Generation Rank           2251 non-null   object 
 4   Capacity Growth Status               2251 non-null   object 
 5   CAIDI w/ Major Events                2251 non-null   float64
 6   CAIDI w/o Major Events               2251 non-null   float64
 7   SAIFI Major Events                   2251 non-null   object 
 8   Utility Annual Energy Savings (MWh)  2251 non-null   object 
 9   Utility Peak Demand Savings (MW)     2251 non-null   object 
 10  Utility Annual Incentive Cost ($)    2251 non-null   object 
 11  Utility Commercial Avg Life (Yrs)  

In [66]:
# Replace 'No IXP' with NaN to exclude it from the count
merged_4['IXP Name'] = merged_4['IXP Name'].replace('No IXP', np.nan)

# Group by 'State' and count unique 'IXP Name' values, then reset index
ixp_counts = merged_4.groupby('State')['IXP Name'].nunique().reset_index()


# Rename the column to 'IXP_Count'
ixp_counts = ixp_counts.rename(columns={'IXP Name': 'IXP_Count'})

# Merge the counts back into the original DataFrame
merged_4 = merged_4.merge(ixp_counts, on='State', how='left')

# Fill NaN values in 'IXP Name' back with 'No IXP'
merged_4['IXP Name'] = merged_4['IXP Name'].fillna('No IXP')

In [67]:
merged_4['IXP_Count'].value_counts()

IXP_Count
27    780
18    294
7     224
11    204
4     152
10    140
5     125
8     104
3      96
2      72
1      60
Name: count, dtype: int64

In [68]:
merged_4.drop(columns=['IXP Name'], inplace=True)
merged_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251 entries, 0 to 2250
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                2251 non-null   object 
 1   Total Generation Rank                2251 non-null   object 
 2   Electric Generation Rank             2251 non-null   object 
 3   Commercial Generation Rank           2251 non-null   object 
 4   Capacity Growth Status               2251 non-null   object 
 5   CAIDI w/ Major Events                2251 non-null   float64
 6   CAIDI w/o Major Events               2251 non-null   float64
 7   SAIFI Major Events                   2251 non-null   object 
 8   Utility Annual Energy Savings (MWh)  2251 non-null   object 
 9   Utility Peak Demand Savings (MW)     2251 non-null   object 
 10  Utility Annual Incentive Cost ($)    2251 non-null   object 
 11  Utility Commercial Avg Life (Y

In [76]:
merged_4.head().sort_values(by='State', ascending=False)

,City,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP_Count
1,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low,2
2,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low,2
3,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low,2
4,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low,2
0,Anchorage,AK,Low,Low,High,Growing,138.8,137.6,High,Unknown,Unknown,Unknown,Unknown,Moderate,1


In [77]:
# Move City column to beginning of dataframe

cols = merged_4.columns.tolist()

# Remove 'City' from current position and add to beginning
cols.remove('City')
cols = ['City'] + cols

# Reorder the DataFrame
merged_4 = merged_4[cols]

# Verify the new column order
print(merged_4.columns)

Index(['City', 'State', 'Total Generation Rank', 'Electric Generation Rank',
       'Commercial Generation Rank', 'Capacity Growth Status',
       'CAIDI w/ Major Events', 'CAIDI w/o Major Events', 'SAIFI Major Events',
       'Utility Annual Energy Savings (MWh)',
       'Utility Peak Demand Savings (MW)', 'Utility Annual Incentive Cost ($)',
       'Utility Commercial Avg Life (Yrs)', 'Regional Electricity Demand',
       'IXP_Count'],
      dtype='object')


### Loading Retail Costs

In [78]:
retail_costs = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Retail_Costs.csv', index_col=0)
retail_costs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1218 entries, 0 to 1238
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NAME            1218 non-null   object
 1   CITY            1218 non-null   object
 2   STATE           1218 non-null   object
 3   Total_MWh_Rank  1218 non-null   object
 4   Purchased_Rank  1218 non-null   object
dtypes: object(5)
memory usage: 57.1+ KB


In [79]:
retail_costs.head()

,NAME,CITY,STATE,Total_MWh_Rank,Purchased_Rank
0,EVERGY METRO,KANSAS CITY,KS,High,High
1,"EVERGY KANSAS SOUTH, INC",TOPEKA,KS,High,High
2,KARNES ELECTRIC COOP INC,KARNES CITY,TX,High,High
3,KAY ELECTRIC COOP,BLACKWELL,OK,Moderate,Moderate
4,FREESTATE ELECTRIC COOP,MCLOUTH,KS,Low,Low


In [80]:
retail_costs = retail_costs.rename(columns={'STATE': 'State', 'NAME': 'Name', 'CITY':'City',
                                'Purchased_Rank': 'Purchased Rank', 'Total_MWh_Rank': 'Total MWh Rank'})
retail_costs['City'].value_counts()

City
COLUMBUS          12
JACKSON            6
AKRON              6
MADISON            5
LEBANON            5
                  ..
MONTE VISTA        1
KINGSTREE          1
SAN DIEGO          1
MOUNT PLEASANT     1
COLDWATER          1
Name: count, Length: 994, dtype: int64

In [81]:
# Check number of states in retail
len(retail_costs['State'].value_counts())

52

In [82]:
retail_costs['State'].value_counts()

# Drop US territories
retail_costs = retail_costs[~retail_costs['State'].isin(['PR', 'GU', 'VI', 'AS', 'CNMI'])]
retail_costs['State'].value_counts()

State
TN    82
TX    75
WI    64
GA    55
NC    47
OH    47
AL    43
IN    43
MN    41
KY    41
MO    41
MS    40
FL    37
WA    37
CA    30
CO    29
OK    29
SC    28
MI    28
AR    26
IL    22
KS    21
LA    21
MA    21
IA    21
NE    20
OR    20
PA    20
AZ    17
VA    17
NY    16
ND    15
SD    15
NM    14
UT    12
ID     9
NV     9
AK     7
MT     7
DE     6
WY     6
CT     6
MD     5
ME     5
NH     5
HI     4
NJ     4
VT     4
RI     3
DC     1
WV     1
Name: count, dtype: int64

In [83]:
len(retail_costs['State'].value_counts())

51

In [84]:
# Check number of states in merged 4

len(merged_4['State'].value_counts())

41

In [85]:
#Lowercase City column
retail_costs['City'] = retail_costs['City'].str.title()
retail_costs['City'].value_counts()

City
Columbus          12
Jackson            6
Akron              6
Madison            5
Lebanon            5
                  ..
Kingstree          1
San Diego          1
Mount Pleasant     1
Livingston         1
Coldwater          1
Name: count, Length: 993, dtype: int64

In [89]:
retail_costs[retail_costs['City'] == 'Columbus']

,Name,City,State,Total MWh Rank,Purchased Rank
79,LOUP RIVER PUBLIC POWER DIST,Columbus,NE,High,High
170,BARTHOLOMEW COUNTY RURAL E M C,Columbus,IN,Moderate,Moderate
205,NEBRASKA PUBLIC POWER DISTRICT,Columbus,NE,High,High
269,OHIO POWER CO,Columbus,OH,High,High
671,WHEELING POWER CO,Columbus,OH,High,High
713,KENTUCKY POWER CO,Columbus,OH,High,High
866,CITY OF COLUMBUS - (MS),Columbus,MS,Moderate,Moderate
868,CITY OF COLUMBUS - (WI),Columbus,WI,Low,Low
888,CORNHUSKER PUBLIC POWER DIST,Columbus,NE,Moderate,Moderate
1031,4-COUNTY ELECTRIC POWER ASSN,Columbus,MS,High,High


In [94]:
# Aggregate retail data at city level before merging
retail_costs_agg = retail_costs.groupby(['State', 'City']).agg({
    'Total MWh Rank': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown',  # Most common rank
    'Purchased Rank': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown',  # Most common rank
    # Add any other columns that need aggregation
}).reset_index()

retail_costs_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123 entries, 0 to 1122
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   State           1123 non-null   object
 1   City            1123 non-null   object
 2   Total MWh Rank  1123 non-null   object
 3   Purchased Rank  1123 non-null   object
dtypes: object(4)
memory usage: 35.2+ KB


In [95]:
retail_costs_agg.groupby(['State', 'City']).size().sort_values(ascending=False).head(30)

State  City         
AK     Anchorage        1
OK     Edmond           1
       Marietta         1
       Lindsay          1
       Kingfisher       1
       Hulbert          1
       Hugo             1
       Hooker           1
       Durant           1
       Oklahoma City    1
       Collinsville     1
       Cleveland        1
       Claremore        1
       Blackwell        1
       Binger           1
       Ada              1
       Norman           1
       Okmulgee         1
TN     Alcoa            1
OK     Woodward         1
OR     Florence         1
       Eugene           1
       Deer Island      1
       Coquille         1
       Clatskanie       1
       Baker City       1
OK     Wilburton        1
       Ponca City       1
       Walters          1
       Vinita           1
dtype: int64

In [99]:
# Merge on both State and City
merged_5 = pd.merge(
    merged_4,
    retail_costs_agg,
    on=['State', 'City'],
    how='left'
)

merged_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251 entries, 0 to 2250
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2251 non-null   object 
 1   State                                2251 non-null   object 
 2   Total Generation Rank                2251 non-null   object 
 3   Electric Generation Rank             2251 non-null   object 
 4   Commercial Generation Rank           2251 non-null   object 
 5   Capacity Growth Status               2251 non-null   object 
 6   CAIDI w/ Major Events                2251 non-null   float64
 7   CAIDI w/o Major Events               2251 non-null   float64
 8   SAIFI Major Events                   2251 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2251 non-null   object 
 10  Utility Peak Demand Savings (MW)     2251 non-null   object 
 11  Utility Annual Incentive Cost 

In [100]:
merged_5[merged_5['Total MWh Rank'].isna()]

,City,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP_Count,Total MWh Rank,Purchased Rank
1,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low,2,NaN,NaN
2,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low,2,NaN,NaN
3,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low,2,NaN,NaN
4,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low,2,NaN,NaN
6,Flagstaff,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate,High,High,High,Moderate,Low,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,Montpelier,VT,Low,Low,Low,Shrinking,316.5,140.0,High,Low,Low,Low,Moderate,Low,1,NaN,NaN
2113,White River Junction,VT,Low,Low,Low,Shrinking,316.5,140.0,High,Low,Low,Low,Moderate,Low,1,NaN,NaN
2115,Montpelier,VT,Low,Low,Low,Shrinking,316.5,140.0,High,Moderate,Low,High,Moderate,Low,1,NaN,NaN
2116,White River Junction,VT,Low,Low,Low,Shrinking,316.5,140.0,High,Moderate,Low,High,Moderate,Low,1,NaN,NaN


In [101]:
merged_5.isna().sum()

City                                     0
State                                    0
Total Generation Rank                    0
Electric Generation Rank                 0
Commercial Generation Rank               0
Capacity Growth Status                   0
CAIDI w/ Major Events                    0
CAIDI w/o Major Events                   0
SAIFI Major Events                       0
Utility Annual Energy Savings (MWh)      0
Utility Peak Demand Savings (MW)         0
Utility Annual Incentive Cost ($)        0
Utility Commercial Avg Life (Yrs)        0
Regional Electricity Demand              0
IXP_Count                                0
Total MWh Rank                         955
Purchased Rank                         955
dtype: int64

## Loading in Emissions Data

In [102]:
emissions = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Emissions.csv', index_col=0)

emissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7511 entries, 0 to 7510
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REPORTING YEAR                   7511 non-null   int64  
 1   FACILITY NAME                    7511 non-null   object 
 2   LATITUDE                         7511 non-null   float64
 3   LONGITUDE                        7511 non-null   float64
 4   CITY                             7511 non-null   object 
 5   COUNTY                           7511 non-null   object 
 6   STATE                            7511 non-null   object 
 7   ZIP CODE                         7511 non-null   int64  
 8   PARENT COMPANIES                 7511 non-null   object 
 9   GHG QUANTITY (METRIC TONS CO2e)  7511 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 645.5+ KB


In [103]:
emissions.head()

,REPORTING YEAR,FACILITY NAME,LATITUDE,LONGITUDE,CITY,COUNTY,STATE,ZIP CODE,PARENT COMPANIES,GHG QUANTITY (METRIC TONS CO2e)
0,2023,(430) Civitas Resources - Permian Basin,39.745220,-104.989197,Denver,Unknown,CO,80202,CIVITAS RESOURCES INC,844548
1,2023,(540) Civitas Resources - Denver Basin,39.744310,-104.988580,Denver,Unknown,CO,80202,CIVITAS RESOURCES INC,887487
2,2023,(540) Civitas Resources - GB - Denver Basin,39.745822,-104.989243,Denver,Unknown,CO,80202,CIVITAS RESOURCES INC,110747
3,2023,121 REGIONAL DISPOSAL FACILITY,33.298570,-96.535860,MELISSA,COLLIN COUNTY,TX,75454,NORTH TEXAS MUNICIPAL WATER DISTRICT,288302
4,2023,15-18565/15-18662,37.274127,-83.239034,Hazard,PERRY COUNTY,KY,40701,CAMBRIAN COAL LLC,122327


In [104]:
# Standardize column names
emissions = emissions.rename(columns={'LATITUDE': 'Latitude',
                                      'LONGITUDE': 'Longitude',
                                      'CITY':'City',
                                      'COUNTY':'County',
                                      'STATE':'State',
                                      'ZIP CODE':'Zip Code',
                                      'GHG QUANTITY (METRIC TONS CO2e)':'GHG (MTCO2)'})


#Drop unnecessary columns
emissions =emissions.drop(columns=['REPORTING YEAR', 'FACILITY NAME', 'PARENT COMPANIES'])


In [105]:
emissions.head()

,Latitude,Longitude,City,County,State,Zip Code,GHG (MTCO2)
0,39.745220,-104.989197,Denver,Unknown,CO,80202,844548
1,39.744310,-104.988580,Denver,Unknown,CO,80202,887487
2,39.745822,-104.989243,Denver,Unknown,CO,80202,110747
3,33.298570,-96.535860,MELISSA,COLLIN COUNTY,TX,75454,288302
4,37.274127,-83.239034,Hazard,PERRY COUNTY,KY,40701,122327


In [106]:
# Standardize  City and County values as title case
emissions['City'] = emissions['City'].str.title()
emissions['County'] = emissions['County'].str.title()
emissions.head()

,Latitude,Longitude,City,County,State,Zip Code,GHG (MTCO2)
0,39.745220,-104.989197,Denver,Unknown,CO,80202,844548
1,39.744310,-104.988580,Denver,Unknown,CO,80202,887487
2,39.745822,-104.989243,Denver,Unknown,CO,80202,110747
3,33.298570,-96.535860,Melissa,Collin County,TX,75454,288302
4,37.274127,-83.239034,Hazard,Perry County,KY,40701,122327


In [107]:
emissions.isna().value_counts()

Latitude  Longitude  City   County  State  Zip Code  GHG (MTCO2)
False     False      False  False   False  False     False          7511
Name: count, dtype: int64

In [108]:
# Group by city and state and take the sum of GHG emissions to create a new column called City Emissions
emissions['City Emissions'] = emissions.groupby(['State', 'City'])['GHG (MTCO2)'].transform('sum')

# Group by state and take the sum of GHG emissions to create a new column called State Emissions
emissions['State Emissions'] = emissions.groupby(['State'])['GHG (MTCO2)'].transform('sum')
emissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7511 entries, 0 to 7510
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Latitude         7511 non-null   float64
 1   Longitude        7511 non-null   float64
 2   City             7511 non-null   object 
 3   County           7511 non-null   object 
 4   State            7511 non-null   object 
 5   Zip Code         7511 non-null   int64  
 6   GHG (MTCO2)      7511 non-null   int64  
 7   City Emissions   7511 non-null   int64  
 8   State Emissions  7511 non-null   int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 586.8+ KB


In [109]:
# Drop Longitude and Latitude columns since we do not need facility locations
emissions = emissions.drop(columns=['Longitude', 'Latitude'])
emissions.head()

,City,County,State,Zip Code,GHG (MTCO2),City Emissions,State Emissions
0,Denver,Unknown,CO,80202,844548,21738141,62456744
1,Denver,Unknown,CO,80202,887487,21738141,62456744
2,Denver,Unknown,CO,80202,110747,21738141,62456744
3,Melissa,Collin County,TX,75454,288302,288302,485547330
4,Hazard,Perry County,KY,40701,122327,164154,65132445


In [110]:
# Group by county and state and take the sum of GHG emissions to create a County Emissions columns
emissions['County Emissions'] = emissions.groupby(['State', 'County'])['GHG (MTCO2)'].transform('sum')
emissions.head()

,City,County,State,Zip Code,GHG (MTCO2),City Emissions,State Emissions,County Emissions
0,Denver,Unknown,CO,80202,844548,21738141,62456744,12704001
1,Denver,Unknown,CO,80202,887487,21738141,62456744,12704001
2,Denver,Unknown,CO,80202,110747,21738141,62456744,12704001
3,Melissa,Collin County,TX,75454,288302,288302,485547330,1381024
4,Hazard,Perry County,KY,40701,122327,164154,65132445,164154


In [111]:
emissions[emissions['Zip Code'] == 0]

,City,County,State,Zip Code,GHG (MTCO2),City Emissions,State Emissions,County Emissions
63,Offshore,Unknown,TX,0,180279,349404,485547330,66853798
504,Offshore,Unknown,LA,0,134118,5733285,145975918,7986796
539,Offshore,Unknown,LA,0,256468,5733285,145975918,7986796
623,Offshore,Unknown,CA,0,35125,35125,99305182,6791172
893,Offshore,Unknown,TX,0,38935,349404,485547330,66853798
...,...,...,...,...,...,...,...,...
7268,Offshore,Unknown,LA,0,194726,5733285,145975918,7986796
7269,Offshore,Unknown,LA,0,53572,5733285,145975918,7986796
7322,Offshore,Unknown,LA,0,12452,5733285,145975918,7986796
7323,Offshore,Unknown,LA,0,8883,5733285,145975918,7986796


In [112]:
# Drop rows where Zip Code is 0
emissions = emissions[emissions['Zip Code'] != 0]

In [113]:
emissions[emissions['County'] == 'Unknown'].head(50)

,City,County,State,Zip Code,GHG (MTCO2),City Emissions,State Emissions,County Emissions
0,Denver,Unknown,CO,80202,844548,21738141,62456744,12704001
1,Denver,Unknown,CO,80202,887487,21738141,62456744,12704001
2,Denver,Unknown,CO,80202,110747,21738141,62456744,12704001
6,Denver,Unknown,CO,80203,27277,21738141,62456744,12704001
7,Denver,Unknown,CO,80203,27277,21738141,62456744,12704001
9,Oklahoma City,Unknown,OK,73102,1166132,13403543,70166208,11908297
11,Houston,Unknown,TX,77079,13382,57927547,485547330,66853798
16,Oklahoma City,Unknown,OK,73142,182531,13403543,70166208,11908297
18,Oklahoma City,Unknown,OK,73102,310278,13403543,70166208,11908297
19,Denver,Unknown,CO,80202,125092,21738141,62456744,12704001


In [114]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [115]:
# Map the zip codes to their counties
import pgeocode    

In [116]:
# Convert zip codes to strings first and create the mapping
emissions['Zip Code'] = emissions['Zip Code'].astype(str)

# Create a clean mapping dictionary
zip_to_county = {}
nomi = pgeocode.Nominatim('us')

# Loop through unique zip codes only
for zip_code in emissions['Zip Code'].unique():
    try:
        location = nomi.query_postal_code(zip_code)
        if location is not None and not pd.isna(location['county_name']):
            county = location['county_name'] + ' County'
            zip_to_county[zip_code] = county
    except:
        continue

# Apply the mapping with a safe approach
emissions.loc[emissions['County'] == 'Unknown', 'County'] = \
    emissions.loc[emissions['County'] == 'Unknown', 'Zip Code'].map(zip_to_county).fillna('Unknown')

In [117]:
emissions[emissions['County'].isna()]

,City,County,State,Zip Code,GHG (MTCO2),City Emissions,State Emissions,County Emissions


In [118]:
emissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7409 entries, 0 to 7510
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   City              7409 non-null   object
 1   County            7409 non-null   object
 2   State             7409 non-null   object
 3   Zip Code          7409 non-null   object
 4   GHG (MTCO2)       7409 non-null   int64 
 5   City Emissions    7409 non-null   int64 
 6   State Emissions   7409 non-null   int64 
 7   County Emissions  7409 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 520.9+ KB


In [119]:
emissions['County'].value_counts()

County
Harris County       295
Dallas County       107
Jefferson County    106
Denver County        82
Oklahoma County      67
                   ... 
Kearney County        1
Buffalo County        1
Riley County          1
Karnes                1
Gratiot County        1
Name: count, Length: 1799, dtype: int64

In [120]:
# Standardize County names
# Function to standardize county names
def standardize_county_name(county):
    # Remove any trailing/leading whitespace
    county = county.strip()
    
    # If it already ends with 'County', return as is
    if county.endswith('County'):
        return county
    # If it ends with 'Parish', return as is (for Louisiana)
    elif county.endswith('Parish'):
        return county
    # If it's not 'Unknown', append 'County'
    elif county != 'Unknown':
        return f"{county} County"
    return county

# Apply the standardization
emissions['County'] = emissions['County'].apply(standardize_county_name)

# Verify the results
print("After standardization:")
print(emissions.groupby(['State', 'County']).size().sort_values(ascending=False).head(30))

After standardization:
State  County               
TX     Harris County            302
       Dallas County            104
CO     Denver County             81
OK     Oklahoma County           69
CA     Los Angeles County        65
       Kern County               58
OK     Tulsa County              44
TX     Midland County            42
CO     Weld County               34
TX     Tarrant County            33
NV     Clark County              31
TX     Jefferson County          30
MI     Wayne County              30
AZ     Maricopa County           30
PA     Allegheny County          29
TX     Travis County             27
CA     San Bernardino County     27
LA     Calcasieu Parish          27
TX     Collin County             26
CA     Contra Costa County       26
AL     Mobile County             25
IL     Cook County               24
TX     Bexar County              24
       Nueces County             23
NM     Lea County                22
CA     San Diego County          21
IL     Will 

In [122]:
# Drop duplicate Counties
emissions = emissions.drop_duplicates(subset=['State', 'City', 'County'])
emissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4131 entries, 0 to 7510
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   City              4131 non-null   object
 1   County            4131 non-null   object
 2   State             4131 non-null   object
 3   Zip Code          4131 non-null   object
 4   GHG (MTCO2)       4131 non-null   int64 
 5   City Emissions    4131 non-null   int64 
 6   State Emissions   4131 non-null   int64 
 7   County Emissions  4131 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 290.5+ KB


In [123]:
# Drop VI, Guam from dataframe
emissions = emissions[~emissions['State'].isin(['VI', 'GU'])]

emissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4122 entries, 0 to 7510
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   City              4122 non-null   object
 1   County            4122 non-null   object
 2   State             4122 non-null   object
 3   Zip Code          4122 non-null   object
 4   GHG (MTCO2)       4122 non-null   int64 
 5   City Emissions    4122 non-null   int64 
 6   State Emissions   4122 non-null   int64 
 7   County Emissions  4122 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 289.8+ KB


In [124]:
# Bin GHG quantity based on EPA and international standards for city-level emissions
def categorize_city_ghg(value):
    if value <= 50000:                    # Small city emitter (<50k MT CO2e/year)
        return 'Low'
    elif value <= 200000:                 # Medium city emitter (50k-200k MT CO2e/year)
        return 'Moderate'
    elif value <= 1000000:                # Large city emitter (200k-1M MT CO2e/year)
        return 'High'            
    else:                                 # Major city emitter (>1M MT CO2e/year)
        return 'Very High'

# Bin GHG quantity based on EPA thresholds for county-level emissions
def categorize_county_ghg(value):
    if value <= 250000:                   # Small county (<250k MT CO2e/year)
        return 'Low'
    elif value <= 1000000:                # Medium county (250k-1M MT CO2e/year)
        return 'Moderate'
    elif value <= 5000000:                # Large county (1M-5M MT CO2e/year)
        return 'High'
    else:                                 # Major county (>5M MT CO2e/year)
        return 'Very High'

# Bin GHG quantity based on state-level emissions standards
def categorize_state_ghg(value):
    if value <= 50000000:                 # Low-emitting state (<50M MT CO2e/year)
        return 'Low'
    elif value <= 100000000:              # Moderate-emitting state (50M-100M MT CO2e/year)
        return 'Moderate'
    elif value <= 200000000:              # High-emitting state (100M-200M MT CO2e/year)
        return 'High'
    else:                                 # Very high-emitting state (>200M MT CO2e/year)
        return 'Very High'

# Apply the categorization
emissions['City Emissions Level'] = emissions['City Emissions'].apply(categorize_city_ghg)
emissions['County Emissions Level'] = emissions['County Emissions'].apply(categorize_county_ghg)
emissions['State Emissions Level'] = emissions['State Emissions'].apply(categorize_state_ghg)

In [125]:
emissions.head()

,City,County,State,Zip Code,GHG (MTCO2),City Emissions,State Emissions,County Emissions,City Emissions Level,County Emissions Level,State Emissions Level
0,Denver,Denver County,CO,80202,844548,21738141,62456744,12704001,Very High,Very High,Moderate
3,Melissa,Collin County,TX,75454,288302,288302,485547330,1381024,High,High,Very High
4,Hazard,Perry County,KY,40701,122327,164154,65132445,164154,Moderate,Low,Moderate
5,Indianapolis,Marion County,IN,46242,64807,2783481,108605519,707549,Very High,Moderate,High
8,Houston,Harris County,TX,77002,286498,57927547,485547330,55445613,Very High,Very High,Very High


In [126]:

# Drop the original numeric columns
emissions = emissions.drop(columns=['City Emissions', 'County Emissions', 'State Emissions', 'GHG (MTCO2)'])
emissions.head()

,City,County,State,Zip Code,City Emissions Level,County Emissions Level,State Emissions Level
0,Denver,Denver County,CO,80202,Very High,Very High,Moderate
3,Melissa,Collin County,TX,75454,High,High,Very High
4,Hazard,Perry County,KY,40701,Moderate,Low,Moderate
5,Indianapolis,Marion County,IN,46242,Very High,Moderate,High
8,Houston,Harris County,TX,77002,Very High,Very High,Very High


In [128]:
# Verify the results

emissions.groupby(['State', 'City', 'County']).size().sort_values(ascending=False).head(30)

State  City          County               
AK     Akutan        Aleutians East County    1
OH     Toronto       Jefferson County         1
OK     Blackwell     Kay County               1
       Bradley       Grady County             1
       Broken Arrow  Wagoner County           1
       Broken Bow    Mccurtain County         1
       Butler        Custer County            1
       Byars         Mcclain County           1
       Calumet       Canadian County          1
       Calvin        Hughes County            1
       Canute        Washita County           1
       Cashion       Kingfisher County        1
                     Logan County             1
       Chouteau      Mayes County             1
       Claremore     Rogers County            1
       Coalgate      Coal County              1
       Cogar         Caddo County             1
       Countyline    Stephens County          1
       Crawford      Roger Mills County       1
       Custer City   Custer County           

In [129]:
merged_5.head()

,City,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),Utility Peak Demand Savings (MW),Utility Annual Incentive Cost ($),Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP_Count,Total MWh Rank,Purchased Rank
0,Anchorage,AK,Low,Low,High,Growing,138.8,137.6,High,Unknown,Unknown,Unknown,Unknown,Moderate,1,High,Moderate
1,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low,2,NaN,NaN
2,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Moderate,Low,Low,Low,2,NaN,NaN
3,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low,2,NaN,NaN
4,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,Low,Low,Moderate,Low,2,NaN,NaN


In [130]:
# Merge emissions with merged 5
merged_6 = pd.merge(merged_5, emissions, on=['State', 'City'], how='left')

merged_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

In [131]:
# Emissions related data that is missing will be filled with unknown
emissions_cols = ['City Emissions Level', 'County Emissions Level', 'State Emissions Level']
merged_6[emissions_cols] = merged_6[emissions_cols].fillna('Unknown')

merged_6.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

In [135]:
merged_6[merged_6.duplicated()]

,City,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),...,Utility Commercial Avg Life (Yrs),Regional Electricity Demand,IXP_Count,Total MWh Rank,Purchased Rank,County,Zip Code,City Emissions Level,County Emissions Level,State Emissions Level
9,Phoenix,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate,High,...,Moderate,Low,8,High,High,Maricopa County,85043,Very High,Very High,Low
10,Phoenix,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate,High,...,Moderate,Low,8,High,High,Maricopa County,85043,Very High,Very High,Low
11,Phoenix,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate,High,...,Moderate,Low,8,High,High,Maricopa County,85043,Very High,Very High,Low
12,Phoenix,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate,High,...,Moderate,Low,8,High,High,Maricopa County,85043,Very High,Very High,Low
13,Phoenix,AZ,High,High,Moderate,Growing,108.6,84.8,Moderate,High,...,Moderate,Low,8,High,High,Maricopa County,85043,Very High,Very High,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,Milwaukee,WI,Moderate,Moderate,Moderate,Shrinking,208.6,114.7,Low,Low,...,Moderate,High,2,High,High,Milwaukee County,53226,High,Moderate,Low
2972,Madison,WI,Moderate,Moderate,Moderate,Shrinking,208.6,114.7,Low,Low,...,Low,High,2,High,High,Dane County,53703,High,Moderate,Low
2973,Milwaukee,WI,Moderate,Moderate,Moderate,Shrinking,208.6,114.7,Low,Low,...,Low,High,2,High,High,Milwaukee County,53226,High,Moderate,Low
2974,Madison,WI,Moderate,Moderate,Moderate,Shrinking,208.6,114.7,Low,Low,...,Moderate,High,2,High,High,Dane County,53703,High,Moderate,Low


# Loading in Broadband Data

In [136]:
broadband = pd.read_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Cleaned Data/Cleaned_Broadband_Data.csv', index_col=0)

broadband.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11385 entries, 25125 to 580965
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   County                  11385 non-null  object
 1   City                    11385 non-null  object
 2   State                   11385 non-null  object
 3   Geog_Area_Type          11385 non-null  object
 4   Gigabit_Fiber_Coverage  11385 non-null  object
 5   Business_Density        11385 non-null  object
dtypes: object(6)
memory usage: 622.6+ KB


In [137]:
broadband.head()

,County,City,State,Geog_Area_Type,Gigabit_Fiber_Coverage,Business_Density
25125,Autauga County,Autauga,AL,Total,Good,Moderate
25155,Baldwin County,Baldwin,AL,Total,Moderate,High
25185,Barbour County,Barbour,AL,Total,Limited,Moderate
25215,Bibb County,Bibb,AL,Total,Moderate,Low
25245,Blount County,Blount,AL,Total,Limited,Moderate


In [138]:
broadband = broadband[broadband['Geog_Area_Type'] == 'Total']

# Drop Geog Area Type
broadband = broadband.drop(columns=['Geog_Area_Type'])

In [139]:
# Count duplicates of City County State
broadband.groupby(['State', 'City', 'County']).size().sort_values(ascending=False).head(30)


State  City                    County                
AK     Aleutians East Borough  Aleutians East Borough    1
OH     Lake                    Lake County               1
       Licking                 Licking County            1
       Logan                   Logan County              1
       Lorain                  Lorain County             1
       Lucas                   Lucas County              1
       Madison                 Madison County            1
       Mahoning                Mahoning County           1
       Marion                  Marion County             1
       Medina                  Medina County             1
       Meigs                   Meigs County              1
       Mercer                  Mercer County             1
       Miami                   Miami County              1
       Monroe                  Monroe County             1
       Montgomery              Montgomery County         1
       Morgan                  Morgan County             1
  

In [142]:
# Merge Broadband with merged_6
merged_7 = pd.merge(merged_6, broadband, on=['State', 'City', 'County'], how='left')

merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

In [143]:
merged_7.head()

,City,State,Total Generation Rank,Electric Generation Rank,Commercial Generation Rank,Capacity Growth Status,CAIDI w/ Major Events,CAIDI w/o Major Events,SAIFI Major Events,Utility Annual Energy Savings (MWh),...,IXP_Count,Total MWh Rank,Purchased Rank,County,Zip Code,City Emissions Level,County Emissions Level,State Emissions Level,Gigabit_Fiber_Coverage,Business_Density
0,Anchorage,AK,Low,Low,High,Growing,138.8,137.6,High,Unknown,...,1,High,Moderate,Anchorage Municipality County,99503,Very High,High,Low,NaN,NaN
1,Anchorage,AK,Low,Low,High,Growing,138.8,137.6,High,Unknown,...,1,High,Moderate,Kenai Peninsula County,99503,Very High,Moderate,Low,NaN,NaN
2,Anchorage,AK,Low,Low,High,Growing,138.8,137.6,High,Unknown,...,1,High,Moderate,Kenai Peninsula Borough County,99503,Very High,High,Low,NaN,NaN
3,Montgomery,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,...,2,NaN,NaN,Montgomery County,36105,Moderate,Low,Moderate,Moderate,High
4,Auburn,AL,High,High,Low,Growing,186.9,122.8,Moderate,Low,...,2,NaN,NaN,Lee County,36830,Low,Low,Moderate,NaN,NaN


In [145]:
merged_7.drop(columns=['City Emissions Level', 'County Emissions Level', 'State Emissions Level'], inplace=True)
merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

In [146]:
# Drop any remaining US territories if they exist
territories = ['PR', 'GU', 'VI', 'AS', 'MP', 'CNMI']
merged_7= merged_7[~merged_7['State'].isin(territories)]

len(merged_7['State'].value_counts())


41

There are 51 states because i included District of Columbia as a state

In [147]:
merged_7.columns

Index(['City', 'State', 'Total Generation Rank', 'Electric Generation Rank',
       'Commercial Generation Rank', 'Capacity Growth Status',
       'CAIDI w/ Major Events', 'CAIDI w/o Major Events', 'SAIFI Major Events',
       'Utility Annual Energy Savings (MWh)',
       'Utility Peak Demand Savings (MW)', 'Utility Annual Incentive Cost ($)',
       'Utility Commercial Avg Life (Yrs)', 'Regional Electricity Demand',
       'IXP_Count', 'Total MWh Rank', 'Purchased Rank', 'County', 'Zip Code',
       'Gigabit_Fiber_Coverage', 'Business_Density'],
      dtype='object')

In [148]:
merged_7 = merged_7.replace('None', 'Unknown')

In [150]:
merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

# Propagating the city level data

In [153]:
# City level columns: 
# Total MWh Rank, Purchased Rank, City Emissions Level

# City-level mappings
city_mappings = {
    'Total MWh Rank': merged_7.groupby(['State', 'City'])['Total MWh Rank'].first(),
    'Purchased Rank': merged_7.groupby(['State', 'City'])['Purchased Rank'].first(),
    'Gigabit_Fiber_Coverage': merged_7.groupby(['State', 'City'])['Gigabit_Fiber_Coverage'].first(),
    'Business_Density': merged_7.groupby(['State', 'City'])['Business_Density'].first()
}

# Fill NaN values using city mappings
for col, mapping in city_mappings.items():
    merged_7[col] = merged_7.apply(
        lambda row: mapping.get((row['State'], row['City'])) if pd.isna(row[col]) or row[col] == 'Unknown' else row[col],
        axis=1
    )

# Verify the results
print("Number of unique cities:", len(merged_7.groupby(['State', 'City'])))
for col in city_mappings.keys():
    print(f"{col}: {len(merged_7[merged_7[col].isna()].groupby(['State', 'City']))} city combinations with NA values")


Number of unique cities: 95
Total MWh Rank: 38 city combinations with NA values
Purchased Rank: 38 city combinations with NA values
Gigabit_Fiber_Coverage: 79 city combinations with NA values
Business_Density: 79 city combinations with NA values


In [154]:
merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

In [155]:
# Drop Zip codes
merged_7.drop(columns=['Zip Code'], inplace=True)
merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

## Propagate the county level data

In [157]:
# Create county-level mappings for emissions, broadband, and disaster data
county_mappings = {
    # Broadband data
    'Gigabit_Fiber_Coverage': merged_7.groupby(['State', 'City', 'County'])['Gigabit_Fiber_Coverage'].first(),
    'Business_Density': merged_7.groupby(['State', 'City', 'County'])['Business_Density'].first()
}

# Fill NaN values using county mappings
for col, mapping in county_mappings.items():
    merged_7[col] = merged_7.apply(
        lambda row: mapping.get((row['State'], row['County'])) 
        if pd.isna(row[col]) or row[col] == 'Unknown' else row[col],
        axis=1
    )

# Verify the results
print("Number of unique counties:", len(merged_7.groupby(['State', 'County'])))
for col in county_mappings.keys():
    print(f"{col}: {len(merged_7[merged_7[col].isna()].groupby(['State', 'County']))} county combinations with NA values")

Number of unique counties: 111
Gigabit_Fiber_Coverage: 89 county combinations with NA values
Business_Density: 89 county combinations with NA values


In [159]:
merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

In [160]:
merged_7.fillna('Unknown', inplace=True)
merged_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   City                                 2979 non-null   object 
 1   State                                2979 non-null   object 
 2   Total Generation Rank                2979 non-null   object 
 3   Electric Generation Rank             2979 non-null   object 
 4   Commercial Generation Rank           2979 non-null   object 
 5   Capacity Growth Status               2979 non-null   object 
 6   CAIDI w/ Major Events                2979 non-null   float64
 7   CAIDI w/o Major Events               2979 non-null   float64
 8   SAIFI Major Events                   2979 non-null   object 
 9   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 10  Utility Peak Demand Savings (MW)     2979 non-null   object 
 11  Utility Annual Incentive Cost 

# Saved merged dataframe

In [161]:
# Save merged_7
merged_7.to_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Data/Merged_Data.csv')


In [162]:
cleaned_df = merged_7.copy()

In [171]:
min_features = cleaned_df[['State','County','IXP_Count','Regional Electricity Demand','Utility Annual Energy Savings (MWh)','SAIFI Major Events', 'CAIDI w/o Major Events','CAIDI w/ Major Events','Commercial Generation Rank','Total Generation Rank']]

min_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                2979 non-null   object 
 1   County                               2979 non-null   object 
 2   IXP_Count                            2979 non-null   int64  
 3   Regional Electricity Demand          2979 non-null   object 
 4   Utility Annual Energy Savings (MWh)  2979 non-null   object 
 5   SAIFI Major Events                   2979 non-null   object 
 6   CAIDI w/o Major Events               2979 non-null   float64
 7   CAIDI w/ Major Events                2979 non-null   float64
 8   Commercial Generation Rank           2979 non-null   object 
 9   Total Generation Rank                2979 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 232.9+ KB


In [172]:
min_features.to_csv('/Users/sabrinasayed/Documents/GitHub/Data_Center_Sites/Data/Minimum_Features.csv')